# Training the sentiment price prediction model

## 0. Setup

Run `$ bash setup.sh` to set up environments and download data.

In [ ]:
# setup environment
!bash setup.sh

In [ ]:
# You can also set up the environment manually:
!conda create --name sent_env --file requirements.

# --or--
# !conda conda env create -f environment.yml

!mkdir -p ./data

# then download the sql files:
!gdown https://drive.google.com/file/d/1YG_AQIbcY2Mi-bKMLN1hff66jDBFeh4O/view?usp=sharing -O ./data/spx_news_sentiment_fundamental.db
!gdown https://drive.google.com/file/d/1C49ElctSD0hPsukQTkioneA7PeWBlMfe/view?usp=sharing -O ./data/spx_news_sentiment_price.db

## 1. Loading data

A dataset class has been implemented already. 

This uses the iterable style of the `Dataset` class.

Initiate the BaseDataset class by passing a Config object, and wrap the kwargs
into the Config object. 

When calling the `__iter__` method using `for data in dataset`, a triple will be returned `(x1, x2, y)`. x1 is the sentiment/price data, x2 is the past financial data up until the train data time, and y is the share price performance during the look_forward period (defaul 5 days, or a trading week). 

The `forward()` method needs to take in (x1, x2), as x2 carries information about the stock's fundamentals and type, but is of a different dimension as x1. An autoencoder model has been implemented and trained to convert the different dimensions of x2 into the same hidden dimensions (and is also useful for dimensionality reduction).

Note x1 and x2 are scaled to between -1 and 1. 

In [1]:
from datasets.datasets import BaseDataset
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from utils import Config

config = Config(mode='train', look_back=100, look_forward=5, num_workers=64)
dataset = BaseDataset(config=config)

## 2. Train autoencoder

In [5]:
from models.autoencoder import AutoEncoder
model = AutoEncoder()
print(model)

AutoEncoder(
  (LSTM_encoder): LSTM(100, 10)
  (Conv1D_encoder): Conv1d(88, 88, kernel_size=(3,), stride=(1,))
  (MaxPool1D_encoder): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Dense_encoder): Linear(in_features=352, out_features=5, bias=True)
  (Dense_decoder): Linear(in_features=5, out_features=352, bias=True)
  (MaxUnpool1D_decoder): MaxUnpool1d(kernel_size=(2,), stride=(2,), padding=(0,))
  (DeConv1D_decoder): ConvTranspose2d(88, 88, kernel_size=(3, 3), stride=(1, 1))
  (LSTM_decoder): LSTM(10, 100)
)


### 2.1 Train loop

In [9]:
import torch.optim as optim
import torch.nn as nn
import torch.functional as F
from utils import padding
from torch.utils.tensorboard import SummaryWriter

lr = 1e-4
epoch = 50
RUNNAME = f"lr={lr}; epoch={epoch}"

writer = SummaryWriter(f'./logs/autoencoder/{RUNNAME}')

input_dims = (88, 100)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

for epoch in range(100):
    running_loss = 0.
    for data in dataset:
        optimizer.zero_grad()
        _, x2, _ = data
        output, _ = model(x2)
        output = padding(x2, direction='left', pad_value=0., 
            repeat=input_dims[-1] - output.shape[-1])
        loss = criterion(output, x2)
        loss.backward()
        running_loss += loss.item()
        optimizer.step()
    writer.add_scalar('loss', running_loss / len(dataset), epoch)
